In [1]:
# Pkg.add( "CUDAdrv")
# Pkg.add("CUDAnative")
# Pkg.add("CuArrays")

In [2]:
#Pkg.add("StatsBase")

In [2]:
using StatsBase: wsample


In [3]:
cd(@__DIR__)

In [4]:
#Pkg.add("Flux")

In [5]:
#Pkg.test("Flux")

In [6]:
using Flux

In [9]:
shakespear = read("100-0.txt", String)
print("B")

B

In [10]:
train = shakespear[3000:end]
length(train)

5793398

In [11]:
text = collect(String(train))
length(text)

5793398

In [40]:
text = replace(text, '$' => "")
text = replace(text, '&' => "")
text = replace(text, '@' => "")
text = replace(text, '*' => "")
text = replace(text, 'æ' => "")
text = replace(text, 'à' => "")
text = replace(text, 'œ' => "")
text = replace(text, 'Æ' => "")
text = replace(text, 'è' => "")
text = replace(text, '%' => "")
text = replace(text, '\\' => "")
text = replace(text, '/'=> "")
text = replace(text, '\'=> "")
text = replace(text, '_' => "")
text = replace(text, '|' => "")
text = replace(text, '[' => "")
text = replace(text, ']'=> "")
text = replace(text, '{' => "")
text = replace(text, '}' => "")
text = replace(text, '`' => "")

text = replace(text, '0' => "")
text = replace(text, '9' => "")
text = replace(text, '8' => "")
text = replace(text, '7' => "")
text = replace(text, '6' => "")
text = replace(text, '5' => "")
text = replace(text, '4' => "")
text = replace(text, '3' => "")
text = replace(text, '2' => "")
text = replace(text, '1' => "")

LoadError: syntax: invalid character literal

In [41]:
alphabet = [unique(text)..., '_']
print(length(alphabet), alphabet)

75Any['\n', '\r', ' ', "", 'F', 'r', 'o', 'm', 'f', 'a', 'i', 'e', 's', 't', 'c', 'u', 'w', 'd', 'n', ',', 'T', 'h', 'b', 'y', '’', 'g', 'v', 'B', 'p', 'l', 'H', ':', '-', 'M', 'k', 'A', 'W', 'P', '.', 'z', ';', 'I', '‘', 'S', 'x', 'L', 'N', 'D', '?', 'O', 'C', 'U', 'q', 'j', 'R', 'Y', '(', ')', 'G', 'V', 'E', 'K', '!', 'J', '—', '“', '”', 'Q', '"', 'X', '\'', 'Z', '`', '\t', '_']

In [42]:
using Flux:onehot
text = map(ch -> onehot(ch, alphabet), text)

5793398-element Array{Flux.OneHotVector,1}:
 [true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, true, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, fals

In [43]:
stop = onehot('_', alphabet)

75-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

In [44]:
N = length(alphabet)
seqlen = 50
nbatch = 50


50

In [45]:
N

75

In [46]:
using Flux: batchseq, chunk
using Base.Iterators: partition
Xs = collect(partition(batchseq(chunk(text,nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch),stop), seqlen))


2318-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 [[false true … false false; true false … false false; … ; false false … false false; false false … false false], [true false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false false … false true; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false 

In [47]:
m = Chain(
    LSTM(N,128),
    LSTM(128,128),
    LSTM(128,128),
    Dense(128,N),
    softmax)

m = gpu(m)

Chain(Recur(LSTMCell(75, 128)), Recur(LSTMCell(128, 128)), Recur(LSTMCell(128, 128)), Dense(128, 75), NNlib.softmax)

In [48]:
using Flux: throttle, crossentropy

function loss(xs, ys)
    l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
    Flux.truncate!(m)
    return l
end

loss (generic function with 1 method)

In [49]:
?Flux.Optimise.ADAM

```
ADAM(params, η = 0.001; β1 = 0.9, β2 = 0.999, ϵ = 1e-08, decay = 0)
```

[ADAM](https://arxiv.org/abs/1412.6980v8) optimiser.


In [50]:
opt = Flux.Optimise.ADAM(params((m),0.01))

#43 (generic function with 1 method)

In [51]:
?Flux.train!

```
train!(loss, data, opt)
```

For each datapoint `d` in `data` computes the gradient of `loss(d...)` through backpropagation and calls the optimizer `opt`.

Takes a callback as keyword argument `cb`. For example, this will print "training" every 10 seconds:

```julia
Flux.train!(loss, data, opt,
            cb = throttle(() -> println("training"), 10))
```

The callback can return `:stop` to interrupt the training loop.

Multiple optimisers and callbacks can be passed to `opt` and `cb` as arrays.


In [52]:

tx, ty = (Xs[5],Ys[5])
evalcb = () -> @show loss(tx,ty)
data = zip(Xs, Ys)


Base.Iterators.Zip2{Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1},Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}}(Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1}[[[true false … false false; false true … false false; … ; false false … false false; false false … false false], [false true … false false; true false … false false; … ; false false … false false; false false … false false], [true false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false false … false true; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [fal

In [ ]:
using Flux: @epochs
@epochs 2 Flux.train!(loss, data, opt, cb = throttle(evalcb,120))

┌ Info: Epoch 1
└ @ Main C:\Users\cwcol\.julia\packages\Flux\UHjNa\src\optimise\train.jl:93


loss(tx, ty) = 215.14980720348072 (tracked)
loss(tx, ty) = 164.9240778801258 (tracked)
loss(tx, ty) = 163.93265845218932 (tracked)
loss(tx, ty) = 163.6258516828179 (tracked)
loss(tx, ty) = 163.69304847655215 (tracked)
loss(tx, ty) = 163.76355489737986 (tracked)
loss(tx, ty) = 163.42605086369326 (tracked)
loss(tx, ty) = 160.43580531021087 (tracked)
loss(tx, ty) = 156.6572945202928 (tracked)
loss(tx, ty) = 151.52217420143936 (tracked)
loss(tx, ty) = 145.32157042727442 (tracked)
loss(tx, ty) = 140.68678365102662 (tracked)
loss(tx, ty) = 137.2386257176148 (tracked)
loss(tx, ty) = 134.55178520730695 (tracked)
loss(tx, ty) = 130.79983020973512 (tracked)
loss(tx, ty) = 128.11909426628145 (tracked)
loss(tx, ty) = 127.08649575703178 (tracked)
loss(tx, ty) = 125.11434160020737 (tracked)
loss(tx, ty) = 123.89236284105145 (tracked)
loss(tx, ty) = 122.03484870736396 (tracked)
loss(tx, ty) = 119.62750535724375 (tracked)
loss(tx, ty) = 117.75042678873506 (tracked)
loss(tx, ty) = 115.6569630336275 (tr

┌ Info: Epoch 2
└ @ Main C:\Users\cwcol\.julia\packages\Flux\UHjNa\src\optimise\train.jl:93


loss(tx, ty) = 91.82584516551529 (tracked)
loss(tx, ty) = 92.15948827052704 (tracked)
loss(tx, ty) = 91.53160082098695 (tracked)
loss(tx, ty) = 92.29026476355067 (tracked)
loss(tx, ty) = 91.37540117855079 (tracked)
loss(tx, ty) = 92.45904656568095 (tracked)
loss(tx, ty) = 92.48409942682636 (tracked)
loss(tx, ty) = 91.9973222000857 (tracked)
loss(tx, ty) = 91.12803751502207 (tracked)
loss(tx, ty) = 91.48191669335435 (tracked)
loss(tx, ty) = 91.9438617010613 (tracked)
loss(tx, ty) = 91.14795529845544 (tracked)
loss(tx, ty) = 90.80027073139676 (tracked)
loss(tx, ty) = 90.15391704247214 (tracked)
loss(tx, ty) = 90.67808064076492 (tracked)
loss(tx, ty) = 90.38793404250781 (tracked)
loss(tx, ty) = 91.04183877680384 (tracked)
loss(tx, ty) = 90.15965276286651 (tracked)
loss(tx, ty) = 90.1927449938352 (tracked)
loss(tx, ty) = 90.69372705045343 (tracked)
loss(tx, ty) = 89.16919361095275 (tracked)
loss(tx, ty) = 90.19281274671799 (tracked)
loss(tx, ty) = 89.13775655357786 (tracked)
loss(tx, ty) =

In [ ]:
function sample(m, alphabet, len; temp = 1)
  m = gpu(m)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println
